# Visual Analytics

## Assignment 3

**Instructor:** Dr. Marco D'Ambros  
**TAs:** Carmen Armenti, Mattia Giannaccari

**Contacts:** marco.dambros@usi.ch, carmen.armenti@usi.ch, mattia.giannaccari@usi.ch

**Due Date:** May 16, 2025 @ 23:55

---
The goal of this assignment is to use **Spark (PySpark)** and **Polars** in Jupyter notebooks.  
The files `trip_data.csv`, `trip_fare.csv`, and `nyc_boroughs.geojson` are available in the provided folder: [Assignment3-data](https://usi365-my.sharepoint.com/:f:/g/personal/armenc_usi_ch/Ejp7sb8QAMROoWe0XUDcAkMBoqUFk-w2Vgroup025NhAww?e=2I7SMC).

You may clean the data as needed; however, please note that specific data cleaning steps will be required in **Exercise 5**. If you choose to clean the data before Exercise 5, make sure to retain the **original dataset** for use with the Polars exercises.

- Use **Spark** to solve **Exercises 1–4**
- Use **Polars** to solve **Exercises 5–8**

You are encouraged to use [Spark window functions](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-window.html) whenever appropriate.

Please name your notebook file as `SurnameName_Assignment3.ipynb`

In [1]:
%pip install pyspark
%pip install polars
%pip install bokeh
%pip install bokeh_sampledata

DEPRECATION: geopolars 0.1.0a4 has a non-standard dependency specifier pyarrow>=4.0.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of geopolars or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: geopolars 0.1.0a4 has a non-standard dependency specifier pyarrow>=4.0.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of geopolars or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, r

## Spark

In [2]:
from sedona.spark import *
from pyspark.sql.functions import col, to_date

### Exercise 1
Join the `trip_data` and `trip_fare` dataframes into one and consider only data on 2013-01-01. Please specify the number of rows obtained after joining the 2 datasets.

The first thing to do is to create a `SparkSession`

In [3]:
sedona_config = (
    SedonaContext.builder()
    .config(
        "spark.jars.packages",
        "org.apache.sedona:sedona-spark-shaded-3.5_2.12:1.7.1,"
        "org.datasyslab:geotools-wrapper:1.7.1-28.5",
    )
    .config(
        "spark.jars.repositories",
        "https://artifacts.unidata.ucar.edu/repository/unidata-all",
    )
    .getOrCreate()
)

spark = SedonaContext.create(sedona_config)

spark.conf.set("spark.sql.shuffle.partitions", 400)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.sparkContext.setLogLevel("ERROR")

25/05/12 11:57:21 WARN Utils: Your hostname, USILU-16210.local resolves to a loopback address: 127.0.0.1; using 10.21.16.27 instead (on interface en0)
25/05/12 11:57:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1
Ivy Default Cache set to: /Users/usi/.ivy2/cache
The jars for the packages stored in: /Users/usi/.ivy2/jars
org.apache.sedona#sedona-spark-shaded-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4417fdd0-334a-49c9-9d27-c53fc6a96091;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/anaconda3/envs/VS-assignment3/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-shaded-3.5_2.12;1.7.1 in central
	found org.datasyslab#geotools-wrapper;1.7.1-28.5 in central
:: resolution report :: resolve 124ms :: artifacts dl 10ms
	:: modules in use:
	org.apache.sedona#sedona-spark-shaded-3.5_2.12;1.7.1 from central in [default]
	org.datasyslab#geotools-wrapper;1.7.1-28.5 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-4417fdd0-334a-49c9-9d27-c53fc6a96091
	confs: [default]
	0 artifacts copied, 2 already retrieved (0kB/12ms)
25/05/12 11:57:22 WARN NativeCodeLoader: Unable to l

In [4]:
trip_data_df = spark.read.csv('./data/trip_data.csv', header=True, inferSchema=True)
trip_fare_df = spark.read.csv('./data/trip_fare.csv', header=True, inferSchema=True)

trip_data_df = trip_data_df.select([col(c).alias(c.strip()) for c in trip_data_df.columns])
trip_fare_df = trip_fare_df.select([col(c).alias(c.strip()) for c in trip_fare_df.columns])

Before joining the 2 dataframe I am going to filter the datas by the given data.

Since we have 2 temporal metric I am going to filter on both in order to avoid `pickup_datetime` being 2012-12-31 while `dropoff_datetime` is 2013-01-01 and in similar way I want to avoid `pickup_datetime` being 2013-01-01 while `dropoff_datetime` is 2013-01-02

In [5]:
trip_data_filtered_df = trip_data_df.filter(
    (to_date("pickup_datetime") == "2013-01-01") &
    (to_date("dropoff_datetime") == "2013-01-01")
)

trip_fare_filtered_df = trip_fare_df.filter(
    (to_date("pickup_datetime") == "2013-01-01")
)

I can now merge the 2 dataframes using the `join` per `medallion`, `hack_license`, `vendor_id` and `pickup_datetime` and print the number of rows.

In [6]:
trip_filtered_df = trip_data_filtered_df.join(
    trip_fare_filtered_df,
    on=["medallion", "hack_license", "vendor_id", "pickup_datetime"],
    how="inner"
)

print(f"The number of rows are: {trip_filtered_df.count()}")

The number of rows are: 410816


### Exercise 2
Provide a graphical representation to compare the average fare amount for trips _within_ and _across_ all the boroughs. You may want to have a look at: https://docs.bokeh.org/en/latest/docs/user_guide/topics/categorical.html#categorical-heatmaps

In [7]:
from pyspark.sql.functions import max as spark_max, min as spark_min, expr

from bokeh.plotting import figure, show, reset_output, output_notebook
from bokeh.models import BasicTicker, PrintfTickFormatter
from bokeh.transform import linear_cmap

reset_output()
output_notebook()

Loading BokehJS ...

As first thing I need to import the `geojson` about the NYC borough.

I open the `GeoJSON` using [`Apache-Sedona`](https://sedona.apache.org/latest/tutorial/files/geojson-sedona-spark/). From the file I only import the `POLYGON`s and the `borough`s.

In [8]:
borough_geo = (spark.read.format("geojson")
    .option("multiLine", "true")
    .load("data/nyc-boroughs.geojson")
    .selectExpr("explode(features) as features")
    .select("features.*")
    .withColumn("borough", expr("properties['borough']")).drop("properties").drop("type").drop("id")
    .withColumnRenamed("geometry", "borough_geom")
)
borough_geo

borough_geom,borough
POLYGON ((-74.050...,Staten Island
POLYGON ((-74.053...,Staten Island
POLYGON ((-74.159...,Staten Island
POLYGON ((-74.082...,Staten Island
POLYGON ((-73.836...,Queens
POLYGON ((-73.813...,Queens
POLYGON ((-73.827...,Queens
POLYGON ((-73.826...,Queens
POLYGON ((-73.832...,Queens
POLYGON ((-73.794...,Queens


Before being able to `spatial_join` on both (`pickup` and `dropoff`) I have to cast the coordinates as a `POINT`.

Then I register the new dataframe and the borough `GeoJSON` in order to execute the `spatial join` on both metrics at the same time.

In [9]:
trips = trip_filtered_df \
    .withColumn("pickup_geom",  expr("ST_Point(CAST(pickup_longitude AS Decimal(24,20)), CAST(pickup_latitude  AS Decimal(24,20)))")) \
    .withColumn("dropoff_geom", expr("ST_Point(CAST(dropoff_longitude AS Decimal(24,20)), CAST(dropoff_latitude AS Decimal(24,20)))"))

trips.createOrReplaceTempView("trips")
borough_geo.createOrReplaceTempView("boroughs")

# 3. Now your SQL will see them:
trips = spark.sql("""
    SELECT
        t.*,
        bp.borough AS pickup_borough,
        bd.borough AS dropoff_borough
    FROM trips t
    LEFT JOIN boroughs bp
        ON ST_Contains(bp.borough_geom, t.pickup_geom)
    LEFT JOIN boroughs bd
        ON ST_Contains(bd.borough_geom, t.dropoff_geom)
""")

trip_filtered_df = trips.drop("pickup_geom").drop("dropoff_geom")
trip_filtered_df


medallion,hack_license,vendor_id,pickup_datetime,rate_code,store_and_fwd_flag,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,pickup_borough,dropoff_borough
00005007A9F30E289...,43468C5D35F828693...,CMT,2013-01-01 07:04:33,1,N,2013-01-01 07:16:03,1,689,3.1,-73.997002,40.732533,-73.970032,40.764423,CRD,12.0,0.0,0.5,2.5,0.0,15.0,Manhattan,Manhattan
00005007A9F30E289...,43468C5D35F828693...,CMT,2013-01-01 08:03:10,1,N,2013-01-01 08:06:41,1,211,1.3,-73.976471,40.743862,-73.985283,40.728035,CSH,6.0,0.0,0.5,0.0,0.0,6.5,Manhattan,Manhattan
00005007A9F30E289...,43468C5D35F828693...,CMT,2013-01-01 12:11:15,1,N,2013-01-01 12:26:47,1,931,8.3,-73.874786,40.774025,-73.970665,40.749886,CRD,24.0,0.0,0.5,5.86,4.8,35.16,Queens,Manhattan
00005007A9F30E289...,43468C5D35F828693...,CMT,2013-01-01 14:01:48,1,N,2013-01-01 14:11:06,1,558,2.3,-73.953491,40.785721,-73.967339,40.759228,CSH,10.0,0.0,0.5,0.0,0.0,10.5,Manhattan,Manhattan
00005007A9F30E289...,A9AE329EA1138052D...,CMT,2013-01-01 02:43:02,1,N,2013-01-01 03:00:27,1,1045,3.7,-73.953056,40.78009,-73.993317,40.758999,CRD,15.5,0.5,0.5,3.3,0.0,19.8,Manhattan,Manhattan
00005007A9F30E289...,C72A773829ED990AF...,CMT,2013-01-01 18:33:03,1,N,2013-01-01 18:38:18,1,315,1.4,-73.958992,40.780998,-73.950035,40.795525,CSH,6.5,0.0,0.5,0.0,0.0,7.0,Manhattan,Manhattan
000318C2E3E638158...,91CE3B3A2F548CD8A...,VTS,2013-01-01 18:22:00,1,NULL,2013-01-01 18:27:00,5,300,1.17,-73.982239,40.773361,-73.991432,40.760235,CRD,6.0,0.0,0.5,1.5,0.0,8.0,Manhattan,Manhattan
000351EDC735C0792...,9413377237F83B3FE...,CMT,2013-01-01 02:02:24,1,N,2013-01-01 02:06:30,2,246,0.7,-73.946609,40.792274,-73.936211,40.794765,CSH,5.0,0.5,0.5,0.0,0.0,6.0,Manhattan,Manhattan
000351EDC735C0792...,9413377237F83B3FE...,CMT,2013-01-01 02:37:23,1,N,2013-01-01 02:39:21,2,118,0.4,-73.980286,40.751598,-73.974274,40.750317,CRD,3.5,0.5,0.5,3.0,0.0,7.5,Manhattan,Manhattan
0009986BDBAB2F9A1...,44CED38841518B1FB...,CMT,2013-01-01 01:10:38,1,N,2013-01-01 01:23:30,1,771,5.3,-73.950111,40.780079,-73.982079,40.722294,CSH,17.5,0.5,0.5,0.0,0.0,18.5,Manhattan,Manhattan


As last thing I visualize the data.

In [10]:
trip_group_df = trip_filtered_df \
    .select(
        col("pickup_borough"),
        col("dropoff_borough"),
        col("fare_amount")
    ) \
    .filter(
        (col("pickup_borough").isNotNull()) &
        (col("dropoff_borough").isNotNull())
    ) \
    .groupBy(['pickup_borough', 'dropoff_borough']) \
    .avg('fare_amount') \
    .withColumnRenamed('avg(fare_amount)', 'avg_fare')

pickup_borough = borough_geo.select('borough').distinct().collect()
dropoff_borough = borough_geo.select('borough').distinct().collect()

pickup_borough = sorted([row.borough for row in pickup_borough])
dropoff_borough = sorted([row.borough for row in dropoff_borough])

min_value = trip_group_df.agg(spark_min('avg_fare')).collect()[0][0]
max_value = trip_group_df.agg(spark_max('avg_fare')).collect()[0][0]

colors = ["#03045e", "#023e8a", "#0077b6", "#0096c7", "#00b4d8", "#48cae4", "#90e0ef", "#ade8f4", "#caf0f8"]

TOOLS = "hover"
TOOLTIPS = [
    ('Pickup Borough', '@pickup_borough'),
    ('Dropoff Borough', '@dropoff_borough'),
    ('Average Fare Amount', '@avg_fare{0.2f}')
]

p = figure(title="Average Fare Amount for Pickup and Dropoff Boroughs",
           x_range=pickup_borough, y_range=dropoff_borough,
           x_axis_location="above", width=900, height=400,
           tools=TOOLS, toolbar_location='below', tooltips=TOOLTIPS)

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0

r = p.rect(x="pickup_borough", y="dropoff_borough", width=1, height=1, source=trip_group_df.toPandas(),
           fill_color=linear_cmap("avg_fare", colors[0:7][::-1], low=min_value, high=max_value),
           line_color=None)

p.add_layout(r.construct_color_bar(
    major_label_text_font_size="7px",
    ticker=BasicTicker(desired_num_ticks=len(colors)),
    formatter=PrintfTickFormatter(format="%d$"),
    label_standoff=6,
    border_line_color=None,
    padding=5,
), 'right')

show(p)

### Exercise 3
Consider only Manhattan, Bronx and Brooklyn boroughs. Then create a dataframe that shows the total number of trips *within* the same borough and *across* all the other boroughs mentioned before (Manhattan, Bronx, and Brooklyn) where the passengers are more or equal than 3.

For example, for Manhattan borough you should consider the total number of the following trips:
- Manhattan → Manhattan
- Manhattan → Bronx
- Manhattan → Brooklyn

You should then do the same for Bronx and Brooklyn boroughs.

In [11]:
from pyspark.sql.functions import desc

To create the required dataframe I have to execute the following steps:
- Filter on the required `borough`s and the `min_passenger`
- Group per both `pickup` and `dropoff` `borough`s
- Aggregate on the `count` of the rows (-> count the number of `trips`)
- Ordering it on both `pickup` and `dropoff` `borough`s for better legibility reason

In [12]:
boroughs = ['Manhattan', 'Brooklyn', 'Bronx']
min_passenger = 3

trip_group_bp_df = trip_filtered_df \
    .filter(
        (col('pickup_borough').isin(boroughs)) &
        (col('dropoff_borough').isin(boroughs)) &
        (col('passenger_count') >= min_passenger)
    ) \
    .groupBy(['pickup_borough', 'dropoff_borough']) \
    .count() \
    .withColumnRenamed('count', 'trip_count') \
    .orderBy(desc('pickup_borough'), desc('dropoff_borough'))

trip_group_bp_df

pickup_borough,dropoff_borough,trip_count
Manhattan,Manhattan,62391
Manhattan,Brooklyn,2762
Manhattan,Bronx,527
Brooklyn,Manhattan,1326
Brooklyn,Brooklyn,2014
Brooklyn,Bronx,10
Bronx,Manhattan,55
Bronx,Brooklyn,2
Bronx,Bronx,103


### Exercise 4
Create a dataframe where each row represents a driver, and there is one column per borough.
For each driver-borough, the dataframe provides the maximum number of consecutive trips
for the given driver, within the given borough. Please consider only trips which were payed by card. 

For example, if for driver A we have (sorted by time):
- Trip 1: Bronx → Bronx
- Trip 2: Bronx → Bronx
- Trip 3: Bronx → Manhattan
- Trip 4: Manhattan → Bronx.
    
The maximum number of consecutive trips for Bronx is 2.

In [13]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, when

First of all I have to sort the dataframe on both `hack_license` and `pickup_datetime` in order to have a linear schedule of the taxi driver.

In [14]:
trip_filtered_df = trip_filtered_df.sort(['hack_license', 'pickup_datetime'], ascending=True)

Now I have, for each `borough`, create a mask for the given requests (same place of `pickup` and `dropoff` and `card` payment).

In order to light it a bit I can remove all the other columns and consider only the minimum required: `hack_license`, `pickup_borough`, `dropoff_borough`, `payment_type`

In [15]:
boroughs = borough_geo.select('borough').distinct().collect()
boroughs = sorted([row.borough for row in boroughs])
payment = 'CRD'

trip_drivers_df = trip_filtered_df \
    .select(
        col('hack_license'),
        col('pickup_borough'),
        col('dropoff_borough'),
        col('payment_type'),
        col('pickup_datetime')
    )

Now I have to create a column for each `borough` in with I have `True` or `False` based on the condition met.

In [16]:
w = Window.partitionBy('hack_license').orderBy('pickup_datetime')

for borough in boroughs:

    trip_drivers_df = trip_drivers_df.withColumn(
        borough,
        when(
            (col('pickup_borough') == borough) &
            (col('dropoff_borough') == borough) &
            (col('payment_type') == payment),
            1
        ).otherwise(0)
    )

    trip_drivers_df = trip_drivers_df.withColumn(
        borough,
        when(
            col(borough) == 1,
            lag(col(borough) + 1, 1, 0).over(w)
        ) \
        .otherwise(0)
    )

trip_drivers_df = trip_drivers_df \
    .groupBy('hack_license') \
    .agg(*[spark_max(borough).alias(borough) for borough in boroughs]) \
    .orderBy('hack_license')


trip_drivers_df

hack_license,Bronx,Brooklyn,Manhattan,Queens,Staten Island
0002555BBE359440D...,0,0,1,1,0
000A4EBF1CEB9C6BD...,0,1,0,0,0
000B8D660A329BBDB...,0,1,2,0,0
000CCA239BFDC0ABE...,0,0,2,0,0
00117D7CCD47D125E...,0,0,2,0,0
0011B1575B9F5398B...,0,1,2,0,0
0012703023AC1788D...,0,0,2,0,0
001C8AAB90AEE49F3...,0,0,2,0,0
0022A86EEFEE75C52...,0,2,1,0,0
0023CF49F98E4F04A...,0,0,2,0,0


## Polars

### Exercise 5

Please work on the merged dataset of trips and fares and perform the following data cleaning tasks:

1. Remove trips with invalid locations (i.e. not in New York City);
3. Remove trips with invalid amounts:
    - Total amount must be greater than zero;
    - Total amount must correspond to the sum of all the other amounts.
5. Remove trips with invalid time:
    - Pick-up before drop-off;
    - Valid duration.

After each data cleaning task, report how many rows where removed. Finally report:
- Are there **duplicate trips**?
- How many trips remain after cleaning?

In [17]:
from typing import Any
from pathlib import Path
from bokeh.models import LinearColorMapper, ColorBar, ColumnDataSource, HoverTool

import polars as pl
import json

In [18]:
tripdata_polar = pl.read_csv('./data/trip_data.csv', infer_schema_length=200, has_header=True)
tripfare_polar = pl.read_csv('./data/trip_fare.csv', infer_schema_length=200, has_header=True)

tripdata_polar = tripdata_polar.rename({col: col.strip() for col in tripdata_polar.columns})
tripfare_polar = tripfare_polar.rename({col: col.strip() for col in tripfare_polar.columns})

tripdata_polar = tripdata_polar.with_columns(
    pl.col("pickup_datetime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S"),
    pl.col("dropoff_datetime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S")
)

tripfare_polar = tripfare_polar.with_columns(
    pl.col("pickup_datetime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S")
)

Before filtering I have to merge the 2 datasets:

In [19]:
trip_polar = tripdata_polar.join(
    tripfare_polar,
    on=['medallion', 'hack_license', 'vendor_id', 'pickup_datetime'],
    how='inner'
)

1. Removing all location that are not New York
2. Checking for correct correspondance of fares
3. Removing invalid trips

To check for valid location I have to get the ranges of `latitude` and `longitude`.

From [NYC.gov](https://www.nyc.gov/assets/tlc/downloads/pdf/proposed_rule_driver_fatigue_rule.pdf): _A Driver must not pick up any passenger(s) for hire in more than 12 hours in total in any 24-hour period._ But for semplicity we filter for hires that are under `12 h`.

In [20]:
def get_min_max_coordinates(
    geojson: dict[str, Any],
) -> tuple[float, float, float, float]:
    """
    Get the min/max coordinates from a geojson file.
    """
    # get all the coordinates from the geojson file
    coordinates: list[tuple[float, float]] = []
    for feature in geojson["features"]:
        coordinates.extend(feature["geometry"]["coordinates"][0])

    min_lon = min([point[0] for point in coordinates])
    max_lon = max([point[0] for point in coordinates])
    min_lat = min([point[1] for point in coordinates])
    max_lat = max([point[1] for point in coordinates])

    return min_lon, max_lon, min_lat, max_lat

In [21]:
nyc_boroughs_geojson_path = Path("./data/nyc-boroughs.geojson")

# read the geojson file
with nyc_boroughs_geojson_path.open("r") as f:
    json_data = f.read()
    nyc_boroughs_geo_data: dict[str, Any] = json.loads(json_data)

# get the min/max coordinates
min_lon, max_lon, min_lat, max_lat = get_min_max_coordinates(nyc_boroughs_geo_data)

In [22]:
trip_polar_filtered = trip_polar.filter(
    pl.col("pickup_longitude") > min_lon,
    pl.col("pickup_longitude") < max_lon,
    pl.col("pickup_latitude") > min_lat,
    pl.col("pickup_latitude") < max_lat,
    pl.col("dropoff_longitude") > min_lon,
    pl.col("dropoff_longitude") < max_lon,
    pl.col("dropoff_latitude") > min_lat,
    pl.col("dropoff_latitude") < max_lat
).filter(
    pl.col("total_amount") > 0,
    (pl.col('total_amount') == (
        pl.col('fare_amount') +
        pl.col('surcharge') +
        pl.col('mta_tax') +
        pl.col('tip_amount') +
        pl.col('tolls_amount')
    ))
).filter(
    pl.col('trip_distance') > 0,
    (pl.col('dropoff_datetime') - pl.col('pickup_datetime')).dt.total_seconds() == pl.col('trip_time_in_secs'),
    (pl.col('dropoff_datetime') - pl.col('pickup_datetime')) < pl.duration(hours=12),
    pl.col('pickup_datetime') < pl.col('dropoff_datetime'),
    pl.col('passenger_count') > 0,
)

trip_polar_filtered

medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
str,str,str,i64,str,datetime[μs],datetime[μs],i64,i64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64
"""89D227B655E5C82AECF13C3F540D4C…","""BA96DE419E711691B9445D6A6307C1…","""CMT""",1,"""N""",2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,-73.978165,40.757977,-73.989838,40.751171,"""CSH""",6.5,0.0,0.5,0.0,0.0,7.0
"""0BD7C8F5BA12B88E0B67BED28BEA73…","""9FD8F69F0804BDB5549F40E9DA1BE4…","""CMT""",1,"""N""",2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.5,-74.006683,40.731781,-73.994499,40.75066,"""CSH""",6.0,0.5,0.5,0.0,0.0,7.0
"""0BD7C8F5BA12B88E0B67BED28BEA73…","""9FD8F69F0804BDB5549F40E9DA1BE4…","""CMT""",1,"""N""",2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.1,-74.004707,40.73777,-74.009834,40.726002,"""CSH""",5.5,1.0,0.5,0.0,0.0,7.0
"""0B57B9633A2FECD3D3B1944AFC7471…","""CCD4367B417ED6634D986F573A552A…","""CMT""",1,"""N""",2013-01-07 12:39:18,2013-01-07 13:10:56,3,1898,10.7,-73.989937,40.756775,-73.86525,40.77063,"""CSH""",34.0,0.0,0.5,0.0,4.8,39.3
"""3349F919AA8AE5DC9C50A3773EA45B…","""7CE849FEF67514F080AF80D990F7EF…","""CMT""",1,"""N""",2013-01-10 15:42:29,2013-01-10 16:04:02,1,1293,3.2,-73.994911,40.723221,-73.971558,40.761612,"""CSH""",15.5,0.0,0.5,0.0,0.0,16.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""F33EF464441839C6F0DABAABBC93B4…","""313F66DD09C308EADA3B307F6B8CF7…","""CMT""",1,"""N""",2013-01-10 10:56:47,2013-01-10 11:05:52,1,545,1.4,-73.97541,40.759106,-73.96183,40.776527,"""CSH""",7.5,0.0,0.5,0.0,0.0,8.0
"""32201027CDC62D654DC3AD9747A07C…","""B8DDB9F8143017E22104050B26C2A6…","""CMT""",1,"""N""",2013-01-05 08:58:18,2013-01-05 09:05:56,1,458,3.2,-73.998901,40.734509,-73.96682,40.770138,"""CSH""",10.5,0.0,0.5,0.0,0.0,11.0
"""B33E71CD9E8FE1BE3B70FEB6E807DD…","""BAF57796E45D921BB23217E17A372F…","""CMT""",1,"""N""",2013-01-06 04:58:23,2013-01-06 05:11:24,1,781,3.3,-73.989029,40.759327,-73.953743,40.770672,"""CSH""",13.0,0.5,0.5,0.0,0.0,14.0


I consider 2 trips being the same iff the have same `hack_license`, `pickup_datetime`, `dropoff_datetime`. I do not consider coordinates to be the same because the result would be in an empty dataframe.

In [23]:
license = ['hack_license']
date = ['pickup_datetime', 'dropoff_datetime']
#position = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']


duplicated_trip = trip_polar_filtered \
    .group_by(license + date) \
    .agg(
        pl.len().alias('duplicated_trip'),
    )

total_duplicated_trip = duplicated_trip \
    .filter(pl.col('duplicated_trip') > 1) \
    .select(
        pl.col('duplicated_trip').sum().alias('total_duplicated_trip')
    ).to_dict()['total_duplicated_trip'][0]

print(f"Total duplicated trip: {total_duplicated_trip}")
print(f"Number of rows removed: {(trip_polar.height - trip_polar_filtered.height)}")

Total duplicated trip: 8
Number of rows removed: 4434723


### Exercise 6

Compute the **total revenue** (total_amount) grouped by:
- Pick-up hour of the day (0–23)
- Passenger count (group >=6 into “6+”)

Create a heatmap where:
- X-axis = hour
- Y-axis = passenger count group
- Cell value = average revenue per trip

As first thing I group as required:

In [24]:
trip_polar_group = trip_polar_filtered \
    .with_columns([
        pl.when(pl.col('passenger_count') < 6)
          .then(pl.col('passenger_count').cast(pl.Utf8))
          .otherwise(pl.lit("6+"))
          .alias('passenger_count'),
        pl.col('pickup_datetime').dt.hour().cast(pl.Int32).alias('pickup_hour')
    ]) \
    .group_by(['pickup_hour', 'passenger_count']) \
    .agg(
        pl.col('total_amount').mean().round(2).alias('avg_amount')
    ) \
    .sort(['pickup_hour'], descending=False)

trip_polar_group

pickup_hour,passenger_count,avg_amount
i32,str,f64
0,"""6+""",14.83
0,"""3""",14.01
0,"""1""",14.64
0,"""5""",14.64
0,"""4""",13.82
…,…,…
23,"""6+""",14.72
23,"""2""",14.45
23,"""4""",13.67


Then I visualize the data.

In [25]:
x_range = [str(h) for h in range(24)]
y_range = sorted(trip_polar_group['passenger_count'].unique(), key=lambda x: int(x.rstrip('+')))

min_value = trip_polar_group['avg_amount'].min()
max_value = trip_polar_group['avg_amount'].max()

colors = ["#03045e", "#023e8a", "#0077b6", "#0096c7", "#00b4d8", "#48cae4", "#90e0ef", "#ade8f4"]

TOOLS = "hover"
TOOLTIPS = [
    ('Pickup Hour', '@pickup_hour'),
    ('passenger_count', '@passenger_count'),
    ('Average Fare Amount', '@avg_amount{0.2f}$')
]

p = figure(title="Average Fare Amount for Pickup Hour and Passenger Count",
           x_axis_label='Pickup Hour',
           y_axis_label='Passenger Count',
           y_range=y_range,
           width=900, height=400,
           tools=TOOLS, toolbar_location=None, tooltips=TOOLTIPS)

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.ticker = list(range(24))
p.x_range.start = -0.5
p.x_range.end = 23.5

color_mapper = LinearColorMapper(palette=colors[::-1], low=min_value, high=max_value)

p.rect(x="pickup_hour", y="passenger_count", width=1, height=1,
       source=trip_polar_group.to_pandas(),
       fill_color=linear_cmap("avg_amount", color_mapper.palette, color_mapper.low, color_mapper.high),
       line_color=None)

color_bar = ColorBar(color_mapper=color_mapper,
                     major_label_text_font_size="7px",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d$"),
                     label_standoff=6,
                     border_line_color=None,
                     padding=5)

p.add_layout(color_bar, 'right')

show(p)

### Exercise 7

Define an "anomalous trip" as one that satisfies at least two of the following:
- Fare per mile is above the 95th percentile
- Tip amount > 100% of fare
- trip_time_in_secs is less than 60 seconds but distance is more than 1 mile

Create a dataframe of anomalous trips and:
- Report how many such trips exist
- Create a scatterplot to visualize the anomaly metrics
- Describe the visualization identifying groups and outliers

I calculate the metrics of:
- `fare_per_mile`
- `average_speed`
- `tip_percentage`

Then I calculate the filters as requested.

Finally I filter the data to meet at least 2 of the 3 conditions

I added the filter as metrics for better visualization later.

In [35]:
from bokeh.models import RadioButtonGroup, CustomJS
from bokeh.layouts import column, row, Spacer

In [27]:
anomalous_trip = trip_polar_filtered \
    .with_columns(
        fare_per_mile=(
            pl.col("fare_amount") / pl.col("trip_distance")
        ).round(2),
        average_speed=(
            pl.col("trip_distance") / (pl.col("trip_time_in_secs") / 3600)
        ).round(2),
        tip_percentage=(
            pl.col("tip_amount") / pl.col("fare_amount")
        ).round(2)
    )

speed_filter = anomalous_trip['fare_per_mile'] > anomalous_trip['fare_per_mile'].quantile(0.95)
tip_filter = anomalous_trip['tip_amount'] > anomalous_trip['fare_amount']
trip_filter = (anomalous_trip['trip_distance'] > 1) & (anomalous_trip['trip_time_in_secs'] < 60)

anomalous_trip = anomalous_trip.with_columns(
    speed_condition=anomalous_trip['fare_per_mile'] > anomalous_trip['fare_per_mile'].quantile(0.95),
    tip_condition=anomalous_trip['tip_amount'] > anomalous_trip['fare_amount'],
    trip_condition=(anomalous_trip['trip_distance'] > 1) & (anomalous_trip['trip_time_in_secs'] < 60)
)

anomalous_trip = anomalous_trip.filter(
    speed_filter + tip_filter + trip_filter >= 2
)


print(f"Number of anomalous trips: {anomalous_trip.height}")

Number of anomalous trips: 1180


In [28]:
number_all_condition = anomalous_trip.filter(
    (pl.col('speed_condition') == True) &
    (pl.col('tip_condition') == True) &
    (pl.col('trip_condition') == True)
).height

print(f"Number of trips with all conditions: {number_all_condition}")

Number of trips with all conditions: 1


Finally, I visualize the data plotting on x axis the `average_speed (mph)` and on y axis `fare_per_hour ($/h)`. The size of the points represent the `tip_percentage`.

In [40]:
min_size, max_size = 5, 30
anomalous_trip = anomalous_trip.with_columns(
    size=(
        (pl.col('tip_percentage') - pl.col('tip_percentage').min()) /
        (pl.col('tip_percentage').max() - pl.col('tip_percentage').min()) *
        (max_size - min_size) + min_size
    ).cast(pl.Int32)
)

source = ColumnDataSource(anomalous_trip.to_dict())

p = figure(title="Anomalous Trips Distribution", 
           x_axis_label='Average Speed (mph)',
           y_axis_label='Fare per Mile ($/mile)',
           tools="pan,wheel_zoom,box_zoom,reset,save",
           width=800, height=600)

scatter = p.scatter(
    x='average_speed',
    y='fare_per_mile',
    size='size',
    source=source,
    fill_color='steelblue',
    fill_alpha=0.9
)

hover = HoverTool(tooltips=[
    ("Tip", "@tip_amount{0.00}"),
    ("Trip Distance", "@trip_distance{0.00} miles"),
    ("Trip Duration", "@trip_time_in_secs{0} seconds"),
    ("Fare per Mile", "@fare_per_mile{0.00} $/mile"),
    ("Average Speed", "@average_speed{0.00} mph"),
])
p.add_tools(hover)

condition_selector = RadioButtonGroup(
    labels=["Condition 1 & 2", "Condition 1 & 3", "Condition 2 & 3", "All Condition", "Any Condition"],
    active=4,
    width=400,
    align="center"
)

filter_code = """
    var condition = this.origin.active;
    var new_data = {};
    var indices = [];

    for (var i = 0; i < original_data.speed_condition.length; i++) {
        var s = original_data.speed_condition[i];
        var t = original_data.tip_condition[i];
        var d = original_data.trip_condition[i];

        var match = false;
        switch (condition) {
            case 0: match = s && t; break;
            case 1: match = s && d; break;
            case 2: match = t && d; break;
            case 3: match = s && t && d; break;
            case 4: match = s || t || d; break;
        }

        if (match) {
            indices.push(i);
        }
    }

    // Update new_data with filtered values
    for (var key in original_data) {
        new_data[key] = [];
        for (var j = 0; j < indices.length; j++) {
            new_data[key].push(original_data[key][indices[j]]);
        }
    }

    // Assign new filtered data to the source
    source.data = new_data;
    source.change.emit();
"""

condition_selector.js_on_event("button_click", CustomJS(
    args=dict(source=source, original_data=source.data),
    code=filter_code
))

p.x_range.start = 0
p.y_range.start = 0

centered_selector = row(Spacer(width=130), condition_selector, Spacer(width=200))
layout = column(p, centered_selector)
show(layout)

#### Condition 1 & 2
Here we can see that we have clearly 2 groups `(1 between 0 and 150 $/h and between 0 and 360 mph)` and `(1 in 250 $/h)` and 4 outliers.

#### Condition 1 & 3
Here we have 1 group and 1 oulier. The group could be subdivided more into 2 subgroups if we bucket the speed in range of 50 mph.

#### Condition 2 & 3
Here we have a group from 0 to 6660 mph and from that point 8 ouliers. But if we consider an outlier based on physics law all the values above 330 mph must be considered as outliers.

#### All Condition
Since I have only 1 element is not possible to determinate groups and outliers.

### Exercise 8
For each driver (hack_license), calculate the **total profit per hour worked**, where:
> profit = 0.7 * (fare_amount + tip_amount) when the trip starts between 7:01 AM and 7:00 PM\
> profit = 0.8 * (fare_amount + tip_amount) when the trip starts between 7:01PM and 7:00 AM

Estimate "hours worked" by summing trip_time_in_secs.

Plot a line chart showing the distribution of average profit per hour **for the top 10% drivers** in terms of total trips.

Which time of day offers **best earning efficiency**?

To create the dataframe need I first have to create some metrics:
- `profit` calculated using the functions above _(obtained in first `with_columns()`)_
- `hour_of_day` to enable the pivoting later on this column  _(obtained in first `with_columns()`)_
- `total_trips`  _(obtained in `agg()`)_
- `total_profit` _(obtained in `agg()`)_
- `profit_per_hour` _(obtained in second `with_columns()`)_

In [30]:
profit_driver = trip_polar_filtered \
    .with_columns(
        pl.when((pl.col('pickup_datetime').dt.hour() > 7) & (pl.col('pickup_datetime').dt.hour() <= 19))
            .then(0.7 * (pl.col('fare_amount') + pl.col('tip_amount')))
            .otherwise(0.8 * (pl.col('fare_amount') + pl.col('tip_amount')))
            .alias('profit'),

        hour_of_day=pl.col('pickup_datetime').dt.hour().alias('pickup_hour')
    ) \
    .group_by('hack_license', 'hour_of_day') \
    .agg(
        (pl.col('trip_time_in_secs').sum()/3600).alias('total_hours'),
        pl.col('profit').sum().alias('total_profit'),
        pl.len().alias('total_trips'),
    ) \
    .with_columns(
        profit_per_hour=(
            pl.col('total_profit') / pl.col('total_hours')
        ).round(2)
    ) \
    .sort('hour_of_day', descending=False)

After that I sort for `total_trips` and select the top `10%` and then I can group per `hour_of_day`.

In [31]:
top_10perc_drivers = profit_driver.select(
    pl.all().top_k_by('total_trips', profit_driver.height * 0.1)
).sort('total_trips', descending=True)

top_10perc_drivers = top_10perc_drivers.group_by('hour_of_day').agg(
    pl.col('total_trips').sum().alias('total_trips'),
    pl.col('total_profit').sum().alias('total_profit'),
    pl.col('profit_per_hour').mean().alias('profit_per_hour')
) \
.sort('hour_of_day', descending=False)

Then I visualize the data showing when there is the change on `profit` formula based on daily hour.

In [32]:
source = ColumnDataSource(top_10perc_drivers)

p = figure(
    title="Daily Distribution of Profit per Hour for Top 10% Drivers",
    x_axis_label="Hour of Day",
    y_axis_label="Profit per Hour ($/h)",
    tools="reset,save",
    width=1000,
    height=600,
    x_range=[0, 23],
)

line_r = p.line(
        x='hour_of_day',
        y='profit_per_hour',
        source=source,
        line_width=2,
    )

hover = HoverTool(tooltips=[
    ("Hour of Day", "@hour_of_day"),
    ("Profit per Hour", "@profit_per_hour{0.00} $/h"),
    ("Total Trips", "@total_trips"),
    ("Total Profit", "@total_profit{0.00} $"),
])
p.add_tools(hover)

p.ygrid.grid_line_color = None
p.xaxis.ticker = list([0, 7, 19, 23])
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.y_range.start = 0

show(p)

In [33]:
print(f"The best earning efficiency is: {round(top_10perc_drivers['profit_per_hour'].max(), 2)} $/h at {top_10perc_drivers['hour_of_day'][top_10perc_drivers['profit_per_hour'].arg_max()]} hour.")


The best earning efficiency is: 74.27 $/h at 5 hour.
